## Singleton Pattern

- __Type:__ Creational
- __Popularity: ★★★★★__
- __Complexity: ★☆☆☆☆__

### Intent:
__Singleton__ is a creational design pattern that ensures a class has only one instance and provides a global point of access to it.

### Problem:
The Singleton pattern solves two problems at the same time, violating the Single Responsibility Principle:
1. **Ensuring a class has just a single instance**: This is useful when you need strict control over global variables or shared resources like a database connection, file system, or configuration settings. For example, multiple parts of your code may need to access a shared database connection, but creating multiple connections could lead to inconsistent data or resource waste.
2. **Providing a global access point to that instance**: Global variables can be accessed from anywhere, which is convenient but breaks encapsulation. The Singleton provides controlled access to the sole instance.

### Solution:
The Singleton pattern makes the default constructor private to prevent other objects from instantiating the Singleton class directly. It provides a static creation method that acts as a constructor, caches the first created object, and returns that cached object for all subsequent calls.

### Diagram:

```mermaid
classDiagram
    class Singleton {
        -static instance: Singleton
        -constructor()
        +static getInstance()
        +businessLogic()
    }
    
    Singleton ..> Singleton : creates
```

## Basic Implementation

In [7]:
class Singleton:
    """Basic Singleton implementation.

    This implementation uses a class variable to store the instance.
    """

    _instance = None  # Class variable to store the single instance

    def __new__(cls):
        """Override the __new__ method to control instance creation."""
        if cls._instance is None:
            # If no instance exists, create one
            cls._instance = super(Singleton, cls).__new__(cls)
            # Initialize instance attributes here if needed
            cls._instance.value = "Initial value"
        return cls._instance

    def some_business_logic(self):
        """Example method to demonstrate the singleton works."""
        return self.value

    def set_value(self, new_value):
        """Method to modify the singleton's state."""
        self.value = new_value

In [8]:
# Client code
if __name__ == "__main__":
    # Create two "instances" of the Singleton
    singleton1 = Singleton()
    singleton2 = Singleton()

    # Check if they are the same instance
    print(f"Are singleton1 and singleton2 the same instance? {singleton1 is singleton2}")

    # Modify the singleton's state through one reference
    singleton1.set_value("Modified value")

    # Check if the modification is visible through both references
    print(f"singleton1 value: {singleton1.some_business_logic()}")
    print(f"singleton2 value: {singleton2.some_business_logic()}")

Are singleton1 and singleton2 the same instance? True
singleton1 value: Modified value
singleton2 value: Modified value


## Thread-Safe Singleton Implementation

In [9]:
import threading


class ThreadSafeSingleton:
    """Thread-safe implementation of the Singleton pattern.

    This implementation uses a lock to prevent race conditions in multi-threaded environments.
    """

    _instance = None
    _lock = threading.Lock()  # Class-level lock for thread safety

    def __new__(cls):
        # Double-checked locking pattern
        if cls._instance is None:
            with cls._lock:  # Acquire lock
                # Check again, another thread might have created the instance
                if cls._instance is None:
                    cls._instance = super(ThreadSafeSingleton, cls).__new__(cls)
                    cls._instance.value = "Initial thread-safe value"
        return cls._instance

    def some_business_logic(self):
        return self.value

    def set_value(self, new_value):
        self.value = new_value

In [10]:
# Client code testing thread safety
def test_singleton_in_thread(thread_name):
    """Function to test singleton in multiple threads."""
    instance = ThreadSafeSingleton()
    print(f"Thread {thread_name}: ID of instance: {id(instance)}")


if __name__ == "__main__":
    # Create threads
    threads = []
    for i in range(5):
        thread = threading.Thread(target=test_singleton_in_thread, args=(f"Thread-{i}",))
        threads.append(thread)

    # Start all threads
    for thread in threads:
        thread.start()

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

Thread Thread-0: ID of instance: 2386226725952
Thread Thread-1: ID of instance: 2386226725952
Thread Thread-2: ID of instance: 2386226725952
Thread Thread-3: ID of instance: 2386226725952
Thread Thread-4: ID of instance: 2386226725952


## Metaclass Singleton Implementation

In [11]:
class SingletonMeta(type):
    """A metaclass that creates a Singleton base class when called.

    Using a metaclass is one of the most elegant ways to implement the Singleton pattern in Python.
    """

    _instances = {}

    def __call__(cls, *args, **kwargs):
        """Called when the instance is 'called' as a function."""
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]


class DatabaseConnection(metaclass=SingletonMeta):
    """Example singleton class using metaclass."""

    def __init__(self, connection_string="default"):
        # This will be called only once
        print(f"Initializing DatabaseConnection with {connection_string}")
        self.connection_string = connection_string

    def connect(self):
        return f"Connected to database using {self.connection_string}"

In [12]:
# Client code with metaclass implementation
if __name__ == "__main__":
    # First 'instance' will initialize the singleton
    db1 = DatabaseConnection("mysql://localhost:3306/db")
    print(db1.connect())

    # Second 'instance' will reuse the existing singleton
    # Note: The connection_string parameter is ignored after first initialization
    db2 = DatabaseConnection("postgres://localhost:5432/db")
    print(db2.connect())

    # Both references point to the same instance
    print(f"Are db1 and db2 the same instance? {db1 is db2}")

Initializing DatabaseConnection with mysql://localhost:3306/db
Connected to database using mysql://localhost:3306/db
Connected to database using mysql://localhost:3306/db
Are db1 and db2 the same instance? True


### Real-world analogies:

1. Government Leadership:
        Many countries have a single leader (president, prime minister) who represents the nation. There can only be one active person in this role at any time. Regardless of who specifically is in office, the title itself ("The President") refers to the current role holder and provides a way for people to access the leader's authority.

2. File System:
        On a computer, there's typically only one file system per storage device. Various programs may interact with the file system through a unified interface, but there's only one instance actually managing the files and directories.

### When to use:
- Use when a class should have exactly one instance accessible to clients
- Use when you need stricter control over global variables
- Use for resources that are expensive to create (database connections, file systems, thread pools)
- Use when you need to coordinate actions across the system through a single point

### Python-specific implementation notes:
- Python's module system can be used as a simple singleton - modules are loaded only once
- Metaclasses offer an elegant way to implement singletons in Python
- Python's `__new__` method allows for controlling instance creation directly
- Thread safety must be considered in concurrent Python applications
- Avoid using global variables instead of singletons, as they lack encapsulation

### Related patterns:
- Factory Method: Often used to create singleton objects
- Facade: May use a singleton to represent a single entry point to a subsystem
- Flyweight: Both patterns aim to reduce the number of objects, but Flyweight shares objects for efficiency while Singleton ensures only one exists
- Monostate (Borg): An alternative to Singleton that focuses on shared state rather than shared instance